In [1]:
%%configure -f 
{
    "conf": {
                "spark.jars.packages":"com.audienceproject:spark-dynamodb_2.11:1.0.4"
            }
}

In [2]:
df = spark.read.parquet('s3://jwyant-tpcds/nested/tpcds/parquet/store_sales')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1588708823081_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df_filtered = df.filter(df.ss_sold_date_sk == 2450816)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_filtered.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ss_customer_sk: integer (nullable = true)
 |-- ss_addr_sk: integer (nullable = true)
 |-- ss_item_sk: integer (nullable = true)
 |-- ss_hdemo_sk: integer (nullable = true)
 |-- ss_promo_sk: integer (nullable = true)
 |-- ss_store_sk: integer (nullable = true)
 |-- ss_sold_time_sk: integer (nullable = true)
 |-- ss_cdemo_sk: integer (nullable = true)
 |-- ss_ticket_number: integer (nullable = true)
 |-- ss_quantity: integer (nullable = true)
 |-- ss_wholesale_cost: double (nullable = true)
 |-- ss_list_price: double (nullable = true)
 |-- ss_sales_price: double (nullable = true)
 |-- ss_ext_discount_amt: double (nullable = true)
 |-- ss_ext_sales_price: double (nullable = true)
 |-- ss_ext_wholesale_cost: double (nullable = true)
 |-- ss_ext_list_price: double (nullable = true)
 |-- ss_ext_tax: double (nullable = true)
 |-- ss_coupon_amt: double (nullable = true)
 |-- ss_net_paid: double (nullable = true)
 |-- ss_net_paid_inc_tax: double (nullable = true)
 |-- ss_net_profit: d

In [5]:
from pyspark.sql.functions import concat_ws
from pyspark.sql.types import StringType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_to_ddb = df_filtered.withColumn('ss_sold_date_sk_ss_ticket_number', concat_ws('-', df_filtered.ss_sold_date_sk.cast(StringType()), df_filtered.ss_ticket_number.cast(StringType())))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
for rec in df_to_ddb.take(5):
    print(rec)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(ss_customer_sk=None, ss_addr_sk=4186, ss_item_sk=280940, ss_hdemo_sk=2561, ss_promo_sk=463, ss_store_sk=None, ss_sold_time_sk=None, ss_cdemo_sk=None, ss_ticket_number=1974871453, ss_quantity=36, ss_wholesale_cost=None, ss_list_price=None, ss_sales_price=33.94, ss_ext_discount_amt=0.0, ss_ext_sales_price=1221.84, ss_ext_wholesale_cost=1322.64, ss_ext_list_price=None, ss_ext_tax=None, ss_coupon_amt=0.0, ss_net_paid=None, ss_net_paid_inc_tax=1295.15, ss_net_profit=-100.8, ss_customer_demographics=None, ss_date_dim=Row(d_date_id=u'AAAAAAAAAIFGFCAA', d_date=datetime.datetime(1998, 1, 2, 0, 0), d_month_seq=1176, d_week_seq=5114, d_quarter_seq=393, d_year=1998, d_dow=5, d_moy=1, d_dom=2, d_qoy=1, d_fy_year=1998, d_fy_quarter_seq=393, d_fy_week_seq=5114, d_day_name=u'Friday', d_quarter_name=u'1998Q1', d_holiday=u'N', d_weekend=u'Y', d_following_holiday=u'Y', d_first_dom=2450815, d_last_dom=2450814, d_same_day_ly=2450451, d_same_day_lq=2450724, d_current_day=u'N', d_current_week=u'N', d_cur

In [9]:
df_to_ddb.repartition(1024, 'ss_sold_date_sk_ss_ticket_number').write.option('tableName', 'store_sales').format('dynamodb').save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from https://10.0.1.199:18888/sessions/9/statements/8 with error payload: {"msg":"requirement failed: Session isn't active."}
